In [1]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer
import torch
from transformers import T5ForConditionalGeneration, AutoConfig
from parent_explore.stagewise_finetune.src.finetune_t5 import SummarizationModule
import pickle
import argparse
import pytorch_lightning as pl
import os

2023-01-09 12:24:19.545653: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-09 12:24:19.545696: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
parser = argparse.ArgumentParser()
parser = pl.Trainer.add_argparse_args(parser)
parser = SummarizationModule.add_model_specific_args(parser, os.getcwd())

In [6]:
inpargs = """
--data_dir=/mnt/data1/prasann/latticegen/lattice-generation/parent_explore/stagewise_finetune/pos/traindata 
--gpus 1 
--learning_rate=3e-5 
--output_dir=/mnt/data1/prasann/latticegen/lattice-generation/parent_explore/stagewise_finetune/src/ignore_tmp
--train_batch_size 3 --eval_batch_size 3 
--max_source_length=120 
--max_target_length=100 
--val_max_target_length=100 
--checkpoint=/mnt/data1/prasann/latticegen/lattice-generation/parent_explore/stagewise_finetune/src/final_model_results/val_avg_bleu=61.9200-step_count=0.ckpt 
--test_max_target_length=100 
--eval_max_gen_length=100 
--model_name_or_path /mnt/data1/prasann/latticegen/lattice-generation/parent_explore/stagewise_finetune/src/final_model_results/best_tfmr 
--task translation 
--early_stopping_patience 15 
--warmup_steps 2 
--do_predict 
--lr_scheduler cosine_w_restarts 
--eval_beams 16
""".split()

In [7]:
args = parser.parse_args(inpargs)

In [8]:
model = SummarizationModule(args)

We have added 0 tokens


In [20]:
dl = model.get_dataloader("test", batch_size=1, shuffle=False)

In [16]:
it = iter(dl)
next(it)

{'input_ids': tensor([[  391, 10665,    12,  1566,    10,     3,    71,   291, 11823,  5735,
              3, 32101,   690,     3, 28060,     3,    71,   291, 11823, 18001,
              3,     2,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1]]),
 'attention_mask': tensor([[1, 

In [3]:
t5modcheck = torch.load("parent_explore/Stage-wise-Fine-tuning/src/final_model_results/val_avg_bleu=61.9200-step_count=0.ckpt")

In [ ]:
t5modcheck['state_dict'].keys()

In [2]:
exs = pd.read_csv("COMET/data/parentqe_train.csv")

In [9]:
ind = 2
print("Source: ", exs.loc[ind]['src'])
print("Hypothesis: ", exs.loc[ind]['mt'])
print("Reference: ", exs.loc[ind]['ref'])
print("Score: ", exs.loc[ind]['score'])

Source:  [["S| A.E Dimitra Efxeinoupolis ", " P| number Of Members ", "O| 1500 ", 3], ["S| A.E Dimitra Efxeinoupolis ", " P| fullname ", "O| A.E Dimitra Efxeinoupolis ", 3], ["S| A.E Dimitra Efxeinoupolis ", " P| ground ", "O| Efxeinoupoli ", 3], ["S| A.E Dimitra Efxeinoupolis ", " P| season ", "O| 2014 ", 3]]
Hypothesis:  The full name of A.E Dimitra Efxeinoupolis is "A.E Dimitra Efxeinoupolis". Their ground is located in Efxeinoupoli and they competed in the 2014 season.
Reference:  A.E. Dimitra Efxeinoupolis has 1500 members, played in the 2014 season and are located in the town of Efxeinoupoli.
Score:  0.4398795783235993


In [ ]:
mod = AutoModel.from_pretrained("parent_explore/plms-graph2text/webnlg-bart-base.ckpt")

In [1]:
# do a sanity check in case our model might actually be a good re-ranker? 

In [2]:
from encode_utils.mt_scores import get_scores_auto

In [21]:
rerdf = pd.read_csv("parent_explore/Stage-wise-Fine-tuning/parent-master/wnlg_rerank_test.csv")

In [4]:
hyps, srcs, refs = list(rerdf['mt']), list(rerdf['src']), list(rerdf['reference'])

In [5]:
parenqe_scos = get_scores_auto(hyps, srcs, refs, 'parentqe', "parentqe")

USING TABLE->TEXT MODEL


Some weights of the model checkpoint at facebook/bart-base were not used when initializing BartModel: ['final_logits_bias']
- This IS expected if you are initializing BartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
Loading weights from /mnt/data1/prasann/latticegen/lattice-generation/COMET/lightning_logs/version_57/checkpoints/epoch=1-step=50000.ckpt.


using correct weight adjustments
using model  bart-base
FREEZE BART SHARED EMBEDS
freeze embeds


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
Predicting DataLoader 0: 100%|██████████████████████████████████████████████████████████████████████| 931/931 [00:39<00:00, 23.80it/s]


TOOK TIME  39.74


In [22]:
rerdf['parentqe'] = parenqe_scos

In [23]:
rerdf

,Unnamed: 0,prec,fsco,recall,reference,src,mt,parentqe
0,0,0.849522,0.852833,0.856169,"Abilene, Texas is served by the Abilene region...",<H> Abilene Regional Airport <R> city Served <...,Abilene Regional Airport serves the city of Ab...,0.659078
1,1,0.738795,0.731484,0.724316,"Abilene, Texas is served by the Abilene region...",<H> Abilene Regional Airport <R> city Served <...,"Abilene Regional Airport serves Abilene, Texas.",0.785484
2,2,0.572253,0.416227,0.327055,"Abilene, Texas is served by the Abilene region...",<H> Abilene Regional Airport <R> city Served <...,Abilene regional airport serves the city of Ab...,0.555692
3,3,1.000000,1.000000,1.000000,"Abilene, Texas is served by the Abilene region...",<H> Abilene Regional Airport <R> city Served <...,Abilene Regional Airport serves the city of Ab...,0.640657
4,4,0.818039,0.836670,0.856169,"Abilene, Texas is served by the Abilene region...",<H> Abilene Regional Airport <R> city Served <...,The Abilene Regional Airport serves the city o...,0.672085
...,...,...,...,...,...,...,...,...
29787,29787,0.770897,0.769829,0.768765,"Alfredo Zitarrosa died in Montevideo, Uruguay....",<H> Uruguay <R> leader Name <T> Raul Fernando ...,"Alfredo Zitarrosa died in Montevideo, Uruguay....",0.581019
29788,29788,0.553238,0.587324,0.625885,"Alfredo Zitarrosa died in Montevideo, Uruguay....",<H> Uruguay <R> leader Name <T> Raul Fernando ...,"Alfredo Zitarrosa died in Montevideo, Uruguay....",0.579860
29789,29789,0.685944,0.727197,0.773730,"Alfredo Zitarrosa died in Montevideo, Uruguay....",<H> Uruguay <R> leader Name <T> Raul Fernando ...,"Alfredo Zitarrosa died in Montevideo, Uruguay....",0.574105
29790,29790,0.740083,0.765308,0.792313,"Alfredo Zitarrosa died in Montevideo, Uruguay....",<H> Uruguay <R> leader Name <T> Raul Fernando ...,"Alfredo Zitarrosa died in Montevideo, Uruguay....",0.579175


In [24]:
rerdf['uid'] = list(range(len(rerdf)))
rerdf['uid'] = (rerdf['uid']/16).round()

In [25]:
def rerank_byval(col):
    N=1
    fsort = rerdf.sort_values(by=['uid', col], ascending=[True, False]).groupby('uid', as_index=False).nth[:N]
    return fsort

In [26]:
rerd = rerank_byval("parentqe")

In [46]:
print("F score", rerd['fsco'].mean())
print("Recall ", rerd['recall'].mean())
print('Precision ', rerd['prec'].mean())

F score 0.7015845281218941
Recall  0.7100264901512193
Precision  0.7325728287399632


In [39]:
rerdf['fsco'].mean()

0.6093371023039469

In [82]:
tmp = rerdf[rerdf['fsco']>rerdf['prec']]
tmp = tmp[tmp['fsco']>tmp['fsco']]

In [78]:
print("times fsco is less than both ", 0.05306)
print("recall is greatest ", .614)
print("precision is greatest ", .3713)


times fsco is less than both  0.05306
recall is greatest  0.614
precision is greatest  0.3713


In [18]:
a = ['apple man', 'apple man', 'b', 'b', 'c']
b = ['apple man', 'b']
c = [1, 2]
adf = pd.DataFrame()
bdf = pd.DataFrame()
adf['col1'] = a
bdf['col1'] = b
bdf['col2'] = c

In [19]:
merged = pd.merge(adf, bdf, on='col1')

In [20]:
merged.apply(lambda x: nlp(x.col1), 1)

0    (apple, man)
1    (apple, man)
2             (b)
3             (b)
dtype: object

In [14]:
import spacy

In [15]:
nlp = spacy.load('en_core_web_sm')

In [26]:
nlp("")